# OPAN5510 Lab Assignment - Joins

This lab focuses on using Polars to perform data joins and aggregations to answer business questions.

# Bike Trips Dataset

## Prerequisites

For this assignment, you'll need to use Polars for data manipulation.

*Insert a code block to import necessary packages (polars)*

In [1]:
# Import necessary packages
import polars as pl

## Load Datasets

Load the `bike_trips.csv` and `bike_weather.csv` files into Polars DataFrames called `trips` and `weather`, respectively.

*Insert code block that reads these files into Polars DataFrames*

In [2]:
# Load the bike trips and weather data
trips = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv", null_values='NA')
weather = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv", null_values=['NA',''])

trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
print(trips.height)
print(weather.height)

78704
733


## Business Question 1: What was the average `duration` of trips that occurred in rainy weather?

#### Part A: Join the trips and weather data frames

The `trips` data represent every ride taken for a bike share company. The `weather` data represent the prevailing weather for a particular day. Join the `trips` and `weather` data together using the `date` column. Name the resulting DataFrame `trips_weather`.

*Insert a code block below that joins `trips` to `weather` using the `date` column.*

In [3]:
# Join trips and weather data
trips_weather = trips.join(weather, on='date')

#### Part B: Calculate the average `duration` of trips in poor weather

Using the `trips_weather` DataFrame, compute the average trip `duration` for days that had weather `events` (i.e. anytime a day had an event of anything but `null`). Name the column for average `duration`: `avg_duration`. The resulting DataFrame should have one row and be named `avg_bad_weather`.

*Insert a code block that shows the computation of the average trip duration for days that had a weather event.*

In [4]:
# Calculate average duration for trips with weather events
avg_bad_weather = trips_weather.filter(
    pl.col('events').is_not_null()
).select(
    pl.col('duration').mean().alias('avg_duration')
)
display(avg_bad_weather)

avg_duration
f64
10.34899


# Baseball Dataset

## Load Data

Load the `Batting.csv`, `People.csv`, and `HallOfFame.csv` datasets into Polars DataFrames.

*Insert a code block to load your datasets*

In [5]:
# Load baseball datasets
Batting = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv", null_values='NA')
People = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv", null_values='NA')
HallOfFame = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv", null_values='NA')


## Question 2: How many home runs (`HR`) were hit by players born in Florida?

#### Part A: Join the `Batting` and `People` DataFrames together

In order to answer this question, you'll need to use the `Batting` and `People` DataFrames. The `Batting` DataFrame has hitting statistics for every season that an individual player has played. The `People` DataFrame represents biographical data about every professional baseball player.

Join the `People` DataFrame into the `Batting` DataFrame so that we can perform analysis on batting statistics using columns from the `People` DataFrame. Name the new DataFrame `stats_w_bio`.

*Insert a code block that joins the `Batting` DataFrame to the `People` DataFrame.*

In [6]:
# Join Batting and People DataFrames
stats_w_bio = Batting.join(People, on='playerID')

#### Part B: Calculate the total number of home runs that were hit by Florida-born players

You would like to perform an analysis about how many home runs (`HR`) were hit by players that were born in the `birthState` of Florida (`FL`). Using the new `stats_w_bio` DataFrame that combines batting statistics with biographical information, compute the total number of home runs (`HR`) hit by players who were born (`birthState`) in Florida (`FL`) and name the new column `total_hr`. The DataFrame should have one row and be named `florida_hr`.

*Insert a block of code that shows the computation of total home runs hit by players born in Florida. The output of this code block should be a DataFrame.*

In [7]:
# Calculate total home runs by Florida-born players
florida_hr = stats_w_bio.filter(
    pl.col('birthState') == 'FL'
).select(
    pl.col('HR').sum().alias('total_hr')
)
display(florida_hr)

total_hr
i64
16225


## Question 3: What are the average number of career Hits (`H`) for Hall of Fame baseball players?

#### Part A: Clean the `HallOfFame` DataFrame

You would like to perform an analysis on the batting statistics of Hall of Fame baseball players. In order to answer this question, you'll need to use the `Batting` and `HallOfFame` DataFrames.

The first thing that you have to do to perform this analysis is to join the `HallOfFame` data into the `Batting` data to understand which players are "Hall of Famers". To join the data correctly, we have to ensure that the `playerID` field is unique in `HallOfFame`. Not every player in the `HallOfFame` DataFrame is a Hall of Famer; their `inducted` field must have the value of `Y` and their `category` field should have the value of `Player`. Name the new DataFrame `hall_inducted`.

*Insert a code block to transform the `HallOfFame` DataFrame to ensure that `playerID` is unique.*

In [8]:
# Clean HallOfFame DataFrame
hall_inducted = HallOfFame.filter(
    (pl.col('inducted') == 'Y') & (pl.col('category') == 'Player')
).select('playerID').unique()

#### Part B: Find the average number of career hits across Hall of Fame players

Join the `hall_inducted` data into the `Batting` data by `playerID`. Aggregate the data to compute the average total hits (`H`) across all Hall of Fame players. To do this, you will first need to calculate the total number of hits for each player and then calculate the average hits across all players. Call the new column `average_hits`. The resulting DataFrame should have one row and be named `hof_hits`.

*Insert a code block that joins the `Batting` and the `hall_inducted` data together and then calculates the average number of career hits (`H`) across all Hall of Fame players. The output of this code block should be a DataFrame.*

In [9]:
# Calculate average career hits for Hall of Fame players
hof_hits = Batting.join(hall_inducted, on='playerID').group_by(
    'playerID'
).agg(
    pl.col('H').sum().alias('total_hits')
).select(
    pl.col('total_hits').mean().alias('average_hits')
)
display(hof_hits)

average_hits
f64
1717.29918


# Task
Complete the following business questions:

1. Calculate the average trip duration during bad weather.
2. Calculate the total home runs hit by players born in Florida.
3. Calculate the average career hits for Hall of Fame players.

Use the following files:
- "trips.csv"
- "weather.csv"
- "Batting.csv"
- "People.csv"
- "HallOfFame.csv"

## Complete business question 1

### Subtask:
Join the `trips` and `weather` DataFrames on the `date` column and name the result `trips_weather`. Filter `trips_weather` to include only rows where the `events` column is not null. Calculate the average `duration` of the filtered data and name the resulting column `avg_duration`. Name the resulting DataFrame `avg_bad_weather`. Display `avg_bad_weather`.


**Reasoning**:
Join the `trips` and `weather` DataFrames on the 'date' column, filter the result to include only rows where the `events` column is not null, calculate the average `duration` for the filtered data, and rename the column to `avg_duration`.

